In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [2]:
driver = webdriver.Firefox(options=options)

In [3]:

urls = ['https://en.wikipedia.org/wiki/2011_ANZ_Championship_season',
        'https://en.wikipedia.org/wiki/2012_ANZ_Championship_season',
        'https://en.wikipedia.org/wiki/2013_ANZ_Championship_season',
        'https://en.wikipedia.org/wiki/2014_ANZ_Championship_season',
        'https://en.wikipedia.org/wiki/2015_ANZ_Championship_season',
        'https://en.wikipedia.org/wiki/2016_ANZ_Championship_season',
        'https://en.wikipedia.org/wiki/2017_Suncorp_Super_Netball_season',
        'https://en.wikipedia.org/wiki/2017_Suncorp_Super_Netball_season',
        'https://en.wikipedia.org/wiki/2018_Suncorp_Super_Netball_season',
        'https://en.wikipedia.org/wiki/2019_Suncorp_Super_Netball_season',
        'https://en.wikipedia.org/wiki/2020_Suncorp_Super_Netball_season'
]
season_round = []
for url in urls:
    driver.get(url)
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, "Regular_season")))
    year_of_season = int(url[30:34])
    soup = BeautifulSoup(driver.page_source)
    scores = soup.find('span',id='Regular_season').parent
    for table in scores.find_next_siblings('table'):
        try:
            if 'mw-collapsible' not in table['class']:
                break
        except:
            break
        game = []
        row = table.find('tr')
        cells = row.find_all('td')
        count = 0
        for element in cells:
            if count == 2:
                if '-' in element.text.strip():
                    game_score = element.text.strip().split('-')
                else:    
                    game_score = element.text.strip().split('–')
                if len(game_score) >1:
                    game.append(int(game_score[0].strip()))
                    game.append(int(game_score[1].strip()))
                count = 3
            else:    
                game.append(element.text.strip())
            count += 1
        game.append(year_of_season)    
        season_round.append(game)

In [ ]:
#Used for testing
# game_urls = []
# url = 'https://en.wikipedia.org/wiki/2012_ANZ_Championship_season'
# driver.get(url)
# WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, "Regular_season")))
# show_elements = driver.find_elements_by_class_name('mw-collapsible-text')
# for element in show_elements:
#     element.click()
# stats_element =  driver.find_elements_by_link_text('Stats')
# year_of_season = int(url[30:34])
# soup = BeautifulSoup(driver.page_source)
# scores = soup.find('span',id='Regular_season').parent
    
    

In [ ]:
cols = ['Date','Team_H','Score_H','Score_A','Team_A','Location','Season']
df = pd.DataFrame(season_round,columns=cols)
for index, row in df.iterrows():
    try:
        int(row[2])
    except:
        df = df.drop(index)

df.to_csv('netball.csv',index=False)